In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import statistics
import math
import datetime
import operator
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score as acc
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from imblearn.over_sampling import SMOTE
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn import metrics
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

C:\Users\Alienware\Anaconda3\envs\EVM\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
def reader(file):
    csv = open(file, 'r',encoding="utf8", errors='ignore')
    data =  pd.read_csv(csv)
    return data

In [3]:
Training_complete = reader('Training Data.csv')

In [4]:
Training_complete = Training_complete.drop('Unnamed: 0',axis = 1)

In [5]:
def multiple_linear_fit(df,replicates):
    for columns in list(df.columns):
        #if not isinstance(df[columns][0],str):
        if columns  not in ['dist1' ,'dist2' ,'card2'] and columns[0] !='D':
            df[columns].fillna(df[columns].mode()[0],inplace = True)
    complete_cols_df = df[df.columns[df.isna().sum() == 0].tolist()]
    #complete_cols_df = complete_cols_df.drop( [x for x in complete_cols_df.columns if isinstance(complete_cols_df[x][0],str)],axis = 1 )
    uncomplete_cols = df.columns[df.isna().any()].tolist()
    for col in uncomplete_cols:
        complete_cols_df[col] = df[col]
        known_part = complete_cols_df[complete_cols_df[col].notna()].as_matrix()
        unknown_part = complete_cols_df[complete_cols_df[col].isna()].as_matrix()
        y = known_part[:, -1]
        X = known_part[:,0:complete_cols_df.shape[1]-2]
       
        lrg = LinearRegression()
        lrg.fit(X, y)
        cnt = 0
        predictedAges = lrg.predict(unknown_part[:,0:complete_cols_df.shape[1]-2])
        for cnt in range(0,replicates-1):
            predictedAges += lrg.predict(unknown_part[:,0:complete_cols_df.shape[1]-2])
            cnt += 1
        df.loc[ (df[col].isna()), col ] = predictedAges/replicates
        complete_cols_df[col] = df[col]
    return df
Training_complete = multiple_linear_fit(Training_complete,5)

In [6]:
Training_complete2 = Training_complete.drop('isFraud',axis = 1) 
Training_complete2['isFraud'] = Training_complete['isFraud']
Training_complete2 = Training_complete2.drop('TransactionID',axis = 1)

In [7]:
Training_complete2.shape

(590540, 359)

In [8]:
data = pd.concat([Training_complete2.iloc[0:150000,:],Training_complete2.iloc[len(Training_complete2)-150000:len(Training_complete),:]])
data

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,isFraud
0,86400,68.500,4,13926,364.973951,150.0,1,142.0,1,315.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
1,86401,29.000,4,2755,404.000000,150.0,2,102.0,1,325.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
2,86469,59.000,4,4663,490.000000,150.0,3,166.0,2,330.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
3,86499,50.000,4,18132,567.000000,150.0,2,117.0,2,476.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
4,86506,50.000,1,4497,514.000000,150.0,2,102.0,1,420.0,...,1,1,123,1,0,1,1,1,954,0
5,86510,49.000,4,5937,555.000000,150.0,3,226.0,2,272.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
6,86522,159.000,4,12308,360.000000,150.0,3,166.0,2,126.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
7,86529,422.500,4,12695,490.000000,150.0,3,226.0,2,325.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
8,86535,15.000,1,2803,100.000000,150.0,3,226.0,2,337.0,...,1,1,98,1,0,0,1,1,1727,0
9,86536,117.000,4,17399,111.000000,150.0,2,224.0,2,204.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0


In [9]:
inconsisent_cols = []
auc_socre = {}
for _ in tqdm(data.columns):
    X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(
    data[[_]+['isFraud']].values[:,:-1],
    data[[_]+['isFraud']].values[:,-1:],
    test_size=0.5,
    random_state=42)
    y_train_1 = y_train_1.ravel()
    y_test_1 = y_test_1.ravel()
    smote = SMOTE(ratio='minority')
    X_sm, y_sm = smote.fit_sample(X_train_1, y_train_1)
    logisticRegr = LogisticRegression()
    logisticRegr.fit(X_sm,y_sm)
    predictions = logisticRegr.predict(X_test_1)
    predictions2 = logisticRegr.predict(X_train_1)
    testing_auc = roc_auc_score(y_test_1,predictions)
    training_auc = roc_auc_score(y_train_1,predictions2)
    auc_socre[training_auc] = testing_auc
    if abs(training_auc - testing_auc) >= 0.15:
        inconsisent_cols.append(_)

100%|██████████| 359/359 [09:11<00:00,  1.54s/it]


In [13]:
auc_socre

{0.5: 0.5,
 0.530974815031686: 0.5325184591586248,
 0.626792118010822: 0.6215236401993982,
 0.5266782749463134: 0.5324635162116742,
 0.507348561558163: 0.507096674842934,
 0.6430000176687733: 0.6418093166837237,
 0.4987023455043541: 0.4940983516961526,
 0.5307770687958158: 0.5321342073882167,
 0.5891059170407943: 0.5971344080039755,
 0.5720006341233572: 0.5712609279697031,
 0.5102240760828468: 0.5108019101470488,
 0.5078039302004871: 0.5096205425216606,
 0.5123017960754063: 0.510216373106857,
 0.6535660164550033: 0.6528716591604942,
 0.5347701874714705: 0.5281815172811223,
 0.5392231221137137: 0.5331508129753962,
 0.5025837357036905: 0.503118440889735,
 0.536215896284445: 0.5297946778910466,
 0.5268471346661117: 0.5271943657214061,
 0.5185487530028177: 0.5129802136925692,
 0.5356172731808582: 0.5334195511003729,
 0.5438124112358069: 0.5369206657336426,
 0.5203398314967893: 0.5187017620714202,
 0.5357047763949238: 0.5328594896191394,
 0.5307790694891162: 0.5235071285337656,
 0.543670572

If we define those features which difference between training_auc and tesing_auc is bigger then 0.15, we find that all features are consistent based on this criteria.

In [ ]:
# If we still have time
inconsisent_cols = []
auc_socre = {}
for _ in tqdm(data.columns):
    X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(
    data[[_]+['isFraud']].values[:,:-1],
    data[[_]+['isFraud']].values[:,-1:],
    test_size=0.5,
    random_state=42)
    y_train_1 = y_train_1.ravel()
    y_test_1 = y_test_1.ravel()
    smote = SMOTE(ratio='minority')
    X_sm, y_sm = smote.fit_sample(X_train_1, y_train_1)
    logisticRegr = LogisticRegression()
    logisticRegr.fit(X_sm,y_sm)
    predictions = logisticRegr.predict(X_test_1)
    predictions2 = logisticRegr.predict(X_train_1)
    testing_auc = roc_auc_score(y_test_1,predictions)
    training_auc = roc_auc_score(y_train_1,predictions2)
    auc_socre[training_auc] = testing_auc
    if abs(training_auc - testing_auc) >= 0.06:
        inconsisent_cols.append(_)

 11%|█▏        | 41/359 [01:03<08:14,  1.56s/it]

In [19]:
inconsisent_cols 

[]